In [1]:
import pandas as pd
import csv
import polars as pl
import pandas as pd
from postalcodes_ca import fsa_codes
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pgeocode
# import geopandas as gpd
pl.enable_string_cache()
import geopandas as gpd

First, I have to assign every postal code to a zone

In [2]:
all_zones = pd.read_csv('ZonalMappingModified3.csv')
all_zones

,ZoneID,CFSAUID,Province,MultiParts,proportion_in_zone,veh_adj,count,Should_prop_be_1,Prop_corrected
0,Alberta_1,T5S,Alberta,NO,0.000000,130425,1,1,1.000000
1,Alberta_1,T5T,Alberta,NO,0.000000,5924,1,1,1.000000
2,Alberta_1,T6M,Alberta,NO,0.000000,2833,1,1,1.000000
3,Alberta_1,T6R,Alberta,NO,0.000000,4613,1,1,1.000000
4,Alberta_10,T0K,Alberta,NO,0.000000,3758,1,1,1.000000
...,...,...,...,...,...,...,...,...,...
3811,Saskatchewan_5,S0N,Saskatchewan,YES,0.004989,11,2,0,0.004989
3812,Saskatchewan_5,S0N,Saskatchewan,YES,0.995011,2123,2,0,0.995011
3813,Saskatchewan_5,S0P,Saskatchewan,YES,0.346111,15,6,0,0.346111
3814,Saskatchewan_5,S0P,Saskatchewan,YES,0.546667,23,6,0,0.546667


In [3]:
all_zones.loc[all_zones['MultiParts'] == 'NO', 'CFSAUID'].value_counts().head(10)

CFSAUID
R0H    4
R0E    3
R0B    3
R0C    2
R3C    2
R5A    2
R5G    2
R0M    2
T6M    1
T6R    1
Name: count, dtype: int64

So I had assumed that if 'MultiParts' == 'NO' then this should be 1 for all, but I guess not. <br>First, I'm going to just examine R0H, which has the most parts

In [4]:
all_zones.loc[all_zones['CFSAUID'] == 'R0H']

,ZoneID,CFSAUID,Province,MultiParts,proportion_in_zone,veh_adj,count,Should_prop_be_1,Prop_corrected
566,Manitoba_5,R0H,Manitoba,NO,0.000361,1,5,0,0.000361
567,Manitoba_5,R0H,Manitoba,NO,0.027076,44,5,0,0.027076
568,Manitoba_5,R0H,Manitoba,NO,0.441516,711,5,0,0.441516
569,Manitoba_5,R0H,Manitoba,NO,0.531047,855,5,0,0.531047


In [5]:
all_zones.loc[all_zones['CFSAUID'] == 'R0E']

,ZoneID,CFSAUID,Province,MultiParts,proportion_in_zone,veh_adj,count,Should_prop_be_1,Prop_corrected
529,Manitoba_3,R0E,Manitoba,NO,0.032837,112,35,0,0.032837
530,Manitoba_3,R0E,Manitoba,NO,0.008501,29,35,0,0.008501
547,Manitoba_4,R0E,Manitoba,NO,0.958662,3273,35,0,0.958662


So multiparts isn't relevant: what matters is I should just select the zone with the highest proportion_in_zone to be the zone for that FSA

In [6]:
zones_selected_df = (
    all_zones
    .loc[all_zones.groupby('CFSAUID')['proportion_in_zone'].idxmax()]
    .reset_index(drop=True)
)
zones_selected_df

,ZoneID,CFSAUID,Province,MultiParts,proportion_in_zone,veh_adj,count,Should_prop_be_1,Prop_corrected
0,NovaScotia_5,B0C,NovaScotia,YES,1.000000,31,59,0,1.000000
1,NovaScotia_5,B0E,NovaScotia,YES,0.884216,450,238,0,0.884216
2,NovaScotia_5,B0H,NovaScotia,YES,0.990344,162,396,0,0.990344
3,NovaScotia_1,B0J,NovaScotia,YES,0.566742,745,1351,0,0.566742
4,NovaScotia_5,B0K,NovaScotia,YES,0.889826,8586,49,0,0.889826
...,...,...,...,...,...,...,...,...,...
1437,BC_17,V9V,British Columbia,NO,0.000000,5214,1,1,1.000000
1438,BC_16,V9W,British Columbia,NO,0.000000,27897,1,1,1.000000
1439,BC_16,V9X,British Columbia,YES,0.999806,713,8,0,0.999806
1440,BC_16,V9Y,British Columbia,YES,0.999929,5746,7,0,0.999929


Now, calculate the search distances

In [7]:
df_processed = pl.read_parquet('VDPs_long_format.parquet')  # full dataset
# df_processed = pl.read_parquet('VDPs_long_format_sampled.parquet')  # 0.1% sample
df_processed = df_processed.with_columns(
    pl.col("seller_fsa").cast(pl.Categorical),
    pl.col("fuel_type").cast(pl.String).str.to_lowercase()
)
df_processed

variable,value,Buyer_FSA,make,model,body_type,condition,year,mileage_50k_bucket,fuel_type,engine,seating_capacity,with_carfax,price_10k_bucket,seller_type,seller_fsa,search_distance_10km_bucket,month_year
str,i16,cat,cat,str,cat,cat,str,cat,str,cat,str,str,str,str,cat,str,datetime[μs]
"""VDPs_201903""",1,"""L6T""","""-""","""-""","""SUV""","""USED""","""0000""","""5""",null,null,null,null,"""0""","""Private""","""L0B""","""6""",2019-03-01 00:00:00
"""VDPs_201905""",1,"""G1A""","""INFINITI""","""G""","""SEDAN""","""USED""","""2011""","""1""",null,null,null,null,"""1""","""Private""","""H7G""","""23""",2019-05-01 00:00:00
"""VDPs_201903""",1,"""N1E""","""INFINITI""","""G""","""COUPE""","""USED""","""2008""","""2""",null,null,null,null,"""1""","""Private""","""V3W""","""328""",2019-03-01 00:00:00
"""VDPs_201904""",1,"""N0A""","""INFINITI""","""G""","""COUPE""","""USED""","""2004""","""2""",null,null,null,null,"""0""","""Dealer""","""V5C""","""338""",2019-04-01 00:00:00
"""VDPs_201905""",6,"""H3L""","""INFINITI""","""G""","""COUPE""","""USED""","""2012""","""1""",null,null,null,null,"""1""","""Dealer""","""J4Y""","""1""",2019-05-01 00:00:00
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""VDPs_202401""",1,"""H2Z""","""LAND ROVER""","""RANGE ROVER""","""SUV""","""USED""","""2023""","""0""","""premium unleaded 91 octane""","""STRAIGHT 6 CYLINDER ENGINE""","""5""",null,"""max""","""Dealer""","""H7T""","""1""",2024-01-01 00:00:00
"""VDPs_202403""",1,"""H2Z""","""LAND ROVER""","""RANGE ROVER""","""SUV""","""USED""","""2023""","""0""","""premium unleaded 91 octane""","""STRAIGHT 6 CYLINDER ENGINE""","""5""",null,"""max""","""Dealer""","""H7T""","""1""",2024-03-01 00:00:00
"""VDPs_202401""",3,"""J3Y""","""LAND ROVER""","""RANGE ROVER SPORT""","""SUV""","""USED""","""2021""","""0""","""premium unleaded 91 octane""","""8 CYLINDER ENGINE""","""5""",null,"""12""","""Dealer""","""H7T""","""2""",2024-01-01 00:00:00


In [8]:
source_col = "fuel_type"  # change if your column name differs

# --- STEP 1: create Standard_fuel_types ---
df_processed = df_processed.with_columns(
    pl.col(source_col)
      .cast(pl.Utf8)
      .str.to_lowercase()
      .replace(  # Polars 1.33 Expr.replace
          {
              'gasoline': 'ICEV', 'gas': 'ICEV', 'gasolinefuel': 'ICEV', 'diesel': 'Diesel',
              'premiumunleaded': 'ICEV', 'regularunleaded': 'ICEV', 'gasolinehybrid': 'HEV',
              'flexible': 'Flexible', 'electric': 'BEV', 'gas/electrichybrid': 'HEV', 'dieselfuel': 'Diesel',
              'gasoline - hybrid': 'HEV', 'premiumunleaded91octane': 'ICEV', 'compressed natural gas': 'AFV',
              'gasoline-hybrid': 'HEV', 'e85-gasoline(flexfuel)': 'Flexible', 'gasoline fuel': 'ICEV', 'compressednaturalgas': 'AFV',
              'propane': 'AFV', 'e85- gasoline(flex fuel)': 'Flexible', 'premium unleaded 91 octane': 'ICEV', 'premium unleaded': 'ICEV',
              'gaseousfuelcompatible': 'AFV', 'regular unleaded': 'ICEV', 'gaseous fuel compatible': 'AFV', 'flexfuelcapability': 'Flexible',
              'diesel fuel': 'Diesel', 'flex fuel capability': 'Flexible', 'naturalgas': 'AFV', 'gas/electric hybrid': 'HEV', 'gasoline hybrid': 'HEV',
              'e85 - gasoline (flex fuel)': 'Flexible', 'natural gas': 'AFV'
          },
          default=pl.col(source_col)  # keep original if no match
      )
      .alias("Standard_fuel_types")
)

# (optional) sanity check:
# print(df_processed.select("Standard_fuel_types").head())

# --- STEP 2: create EV / Non_BEV ---
df_processed = df_processed.with_columns(
    pl.when(pl.col("Standard_fuel_types") == "BEV")
      .then(pl.lit("EV"))
      .when(pl.col("Standard_fuel_types") == "ICEV")
      .then(pl.lit("ICEV"))
      .otherwise(pl.lit("Non BEV, non ICEV"))
      .alias("Fuel_types_few_categories")
)

# Filter out EV only
df_processed = df_processed.filter(pl.col("Fuel_types_few_categories") == "EV")


# --- GROUP ---
df_grouped = (
    df_processed
      .group_by(["Buyer_FSA", "seller_fsa", "Fuel_types_few_categories"])
      .agg(pl.col("value").sum().alias("num_searches"))
)




C:\Users\kikomwen\AppData\Local\Temp\6\ipykernel_24360\3632131087.py:8: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)
  .replace(  # Polars 1.33 Expr.replace


Searches by fuel type

In [9]:
fuel_type_summary = df_grouped.group_by("Fuel_types_few_categories").agg(
    pl.col("num_searches").sum().alias("total_searches")
).to_pandas()
fuel_type_summary['percentage'] = (fuel_type_summary['total_searches'] / fuel_type_summary['total_searches'].sum()) * 100
fuel_type_summary

,Fuel_types_few_categories,total_searches,percentage
0,EV,38086803,100.0


Getting actual search distances

In [10]:
df_fsa_combinations = pl.read_parquet('FSA_distances.parquet')  # read back in

# Make fsa1, fsa2 to categorical
df_fsa_combinations = df_fsa_combinations.with_columns(
    pl.col("fsa1").cast(pl.Categorical),
    pl.col("fsa2").cast(pl.Categorical),
)

In [11]:


# Merge with the main dataframe to get search distances
df_merged = df_grouped.join(df_fsa_combinations, left_on=['Buyer_FSA', 'seller_fsa'], right_on=['fsa1', 'fsa2'], how='left')
df_merged = df_merged.to_pandas()
df_merged

,Buyer_FSA,seller_fsa,Fuel_types_few_categories,num_searches,distance_km
0,N0E,J0G,EV,1,719.054732
1,M6C,L6A,EV,108,18.927604
2,HUD,H4P,EV,1,NaN
3,P3A,L2R,EV,88,397.040637
4,K4A,V7V,EV,9,3556.534600
...,...,...,...,...,...
1039173,L3C,L6A,EV,147,96.692415
1039174,H4X,L1S,EV,4,461.418599
1039175,J0Z,N6H,EV,1,576.051872
1039176,V9C,J0E,EV,1,3823.616470


In [12]:
df_merged_zones = df_merged.merge(
    zones_selected_df[['CFSAUID', 'ZoneID']],
    left_on='Buyer_FSA',
    right_on='CFSAUID',
    how='left'
)
df_merged_zones['distance_km'].fillna(0, inplace=True)    # NA values when origin and destination FSAs are in the same zone
df_merged_zones

C:\Users\kikomwen\AppData\Local\Temp\6\ipykernel_24360\1872398527.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged_zones['distance_km'].fillna(0, inplace=True)    # NA values when origin and destination FSAs are in the same zone


,Buyer_FSA,seller_fsa,Fuel_types_few_categories,num_searches,distance_km,CFSAUID,ZoneID
0,N0E,J0G,EV,1,719.054732,N0E,ON_6
1,M6C,L6A,EV,108,18.927604,M6C,ON_15
2,HUD,H4P,EV,1,0.000000,NaN,NaN
3,P3A,L2R,EV,88,397.040637,NaN,NaN
4,K4A,V7V,EV,9,3556.534600,K4A,ON_24
...,...,...,...,...,...,...,...
1039173,L3C,L6A,EV,147,96.692415,L3C,ON_1
1039174,H4X,L1S,EV,4,461.418599,H4X,QC_4
1039175,J0Z,N6H,EV,1,576.051872,J0Z,QC_16
1039176,V9C,J0E,EV,1,3823.616470,V9C,BC_15


In [13]:
# Group by ZoneID to get average distance weighted by num_searches
df_zone_summary = df_merged_zones.groupby('ZoneID').apply(
    lambda x: pd.Series({
        'total_searches': x['num_searches'].sum(),
        'avg_distance_weighted': np.average(x['distance_km'], weights=x['num_searches'])
    })
).reset_index()
df_zone_summary


C:\Users\kikomwen\AppData\Local\Temp\6\ipykernel_24360\945601972.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_zone_summary = df_merged_zones.groupby('ZoneID').apply(


,ZoneID,total_searches,avg_distance_weighted
0,Alberta_1,122382.0,944.308439
1,Alberta_10,99160.0,907.158447
2,Alberta_11,240463.0,853.933601
3,Alberta_12,267450.0,833.286445
4,Alberta_13,21408.0,1056.101336
...,...,...,...
87,Saskatchewan_1,54007.0,1173.347456
88,Saskatchewan_2,45308.0,1146.764195
89,Saskatchewan_3,56496.0,1220.408904
90,Saskatchewan_4,84956.0,1127.796037


In [14]:
# Save to CSV
df_zone_summary.to_csv('EV_Zonal_Search_Distances_Summary.csv', index=False)